## Example question answering on a web page

In [ ]:
!pip install haystack-ai==2.8.0

In [1]:
from haystack import Pipeline
from haystack.utils import Secret
from haystack.components.fetchers import LinkContentFetcher
from haystack.components.converters import HTMLToDocument
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
import getpass
import os

if "SAMBANOVA_API_KEY" not in os.environ:
    os.environ["SAMBANOVA_API_KEY"] = getpass.getpass("insert your SambaNovaCloud API Key")

In [2]:
fetcher = LinkContentFetcher()
converter = HTMLToDocument()
prompt_template = """
According to the contents of this website:
{% for document in documents %}
  {{document.content}}
{% endfor %}
Answer the given question: {{query}}
Answer:
"""
prompt_builder = PromptBuilder(template=prompt_template)
llm = OpenAIGenerator(
    api_key=Secret.from_env_var("SAMBANOVA_API_KEY"),
    api_base_url="https://api.sambanova.ai/v1/",
    model="Meta-Llama-3.3-70B-Instruct",
    generation_kwargs = {"max_tokens": 1024}
)
pipeline = Pipeline()
pipeline.add_component("fetcher", fetcher)
pipeline.add_component("converter", converter)
pipeline.add_component("prompt", prompt_builder)
pipeline.add_component("llm", llm)

pipeline.connect("fetcher.streams", "converter.sources")
pipeline.connect("converter.documents", "prompt.documents")
pipeline.connect("prompt.prompt", "llm.prompt")

result = pipeline.run({"fetcher": {"urls": ["https://community.sambanova.ai/t/sambanova-cloud-quick-start-guide/194"]},
              "prompt": {"query": "How do I make a request using the SambaNova API?"}})

print(result["llm"]["replies"][0])

To make a request using the SambaNova API, you have two options:

**Option 1: Using a CURL command**

1. Export your API key and URL in a terminal.
2. Run the CURL command with the following parameters:
	* Authorization: Bearer $API_KEY
	* Content-Type: application/json
	* Data: a JSON object containing the model ID, messages, stop sequence, and other options.
3. Send the request to the API using the POST method.

**Option 2: Using the OpenAI client library in Python**

1. Install the OpenAI library using pip.
2. Create a Python file and import the OpenAI library.
3. Initialize the OpenAI client with your API key and base URL.
4. Use the `chat.completions.create` method to make an inference request, passing in the model ID, messages, and other options.
5. Run the Python file to send the request and print the response.

In both cases, you need to replace `<YOUR API KEY>` with your actual API key and use the correct model ID, such as "Meta-Llama-3.1-405B-Instruct".


## Example question answering using RAG

In [3]:
from haystack import Document, Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.document_stores.in_memory import InMemoryDocumentStore

from haystack.components.generators import OpenAIGenerator
import getpass
import os

if "SAMBANOVA_API_KEY" not in os.environ:
    os.environ["SAMBANOVA_API_KEY"] = getpass.getpass("insert your SambaNovaCloud API Key")

document_store = InMemoryDocumentStore()
document_store.write_documents(
    [
        Document(content="The Function-Calling API enables dynamic, agentic workflows by allowing the model to suggest and select function calls based on user input."
                "This feature facilitates flexible agentic workflows that adapt to varied needs."),
        Document(content="Interact with multimodal models directly through the Inference API (OpenAI compatible) and Playground"
                 "for seamless text and image processing."),
        Document(
            content="New Python and Gradio code samples make it easier to build and deploy applications on SambaNova Cloud. These examples simplify"
            "integrating AI models, enabling faster prototyping and reducing setup time."
        ),
    ]
)

template = """
Given only the following information, answer the question.
Ignore your own knowledge.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""

llm = OpenAIGenerator(
    api_key=Secret.from_env_var("SAMBANOVA_API_KEY"),
    api_base_url="https://api.sambanova.ai/v1",
    model="Meta-Llama-3.3-70B-Instruct",
    generation_kwargs = {"max_tokens": 1024}
)

pipe = Pipeline()

pipe.add_component("retriever", InMemoryBM25Retriever(document_store=document_store))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", llm)
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

query = "Functionalities of Sambanova API?"

response = pipe.run({"prompt_builder": {"query": query}, "retriever": {"query": query}})

print(response["llm"]["replies"])

['Based on the given information, the SambaNova Cloud API has the following functionalities:\n\n1. Inference API (OpenAI compatible) for seamless text and image processing.\n2. Function-Calling API for dynamic, agentic workflows, allowing models to suggest and select function calls based on user input.\n3. Playground for interacting with multimodal models directly. \n\nAdditionally, it provides Python and Gradio code samples to simplify integrating AI models, enabling faster prototyping and reducing setup time.']
